### NLP to SQL 

In this project, we'll develop a start-up that can take a non-technical manager's Natural Language question, such as "What counties had the top sales?" and convert that into a SQL query. We will then use that SQL to query the data (which in this example will come from a .csv file we read in with Pandas and set-up a temporary DB in RAM) and then report back the results!

Dataset based on https://www.kaggle.com/datasets/kyanyoga/sample-sales-data

1. Set up API key
2. Read CSV file.
3. Turned that into a in-memory SQL database.
4. Create a table definition from the data frame.
5. Prompt engineering 
6. Input from a non technical user.
7. Combine them to get open AI prompt.
8. Then pass in that prompt to the openai completion create with some usage of stop tokens.
9. Handle that response to clean it up to get out the actual SQL query
10. Execute this code, get the result back and report the results.



### Set up the key, example of prompt and connection to OpenAi

In [64]:
import os

In [65]:
import openai

In [66]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [67]:
prompt = 'Give me two reasons to learn OpenAI API with Python'

In [68]:
response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)

In [69]:
print(response['choices'][0]['text'])



1. OpenAI API with Python provides a powerful set of tools for developing and deploying AI applications. It allows developers to quickly and easily build complex AI models and deploy them in production.

2. OpenAI API with Python is an open source library that provides a wide range of algorithms and tools for machine learning and deep learning. It is easy to use and provides a comprehensive set of features for building and deploying AI applications. Additionally, it is well-documented and supported by a large community of developers.


### Read CSV file and Turned that into a in-memory SQL database

In [70]:
import pandas as pd

In [71]:
df = pd.read_csv("/Users/iris/Downloads/sales_data_sample.csv")

In [72]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [73]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [74]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [75]:
data = df.to_sql(name='Sales',con=temp_db)


2023-04-28 10:07:14,890 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-04-28 10:07:14,891 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-28 10:07:14,892 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-04-28 10:07:14,893 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-28 10:07:14,895 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-28 10:07:14,897 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

In [76]:
with temp_db.connect() as conn:
    result = conn.execute(text("Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1"))

2023-04-28 10:07:21,724 INFO sqlalchemy.engine.Engine Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1
2023-04-28 10:07:21,726 INFO sqlalchemy.engine.Engine [generated in 0.00200s] ()


### Create a prompt

In [97]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [78]:
### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months
# SELECT

In [79]:
def create_table_definition_prompt(df):
    """
    This function returns a prompt that informs GPT that we want to work with SQL Tables
    """

    prompt = '''### sqlite SQL table, with its properties:
#
# Sales({})
#
'''.format(",".join(str(x) for x in df.columns))
    
    return prompt

In [80]:
print(create_table_definition_prompt(df))

### sqlite SQL table, with its properties:
#
# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
#



In [81]:
def prompt_input():
    nlp_text = input("Enter information you want to obtain: ")
    return nlp_text

In [93]:
nlp_text = prompt_input()

Enter information you want to obtain: Can you provide me with the total sales amount for each city?


In [94]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition_prompt(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition+query_init_string

In [95]:
combine_prompts(df, nlp_text)

'### sqlite SQL table, with its properties:\n#\n# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n#\n### A query to answer: Can you provide me with the total sales amount for each city?\nSELECT'

### OpenAI call

In [98]:
response = openai.Completion.create(
        model = 'text-davinci-003',
        prompt = combine_prompts(df, nlp_text),
        temperature = 0,
        max_tokens = 150,
        top_p = 1.0,
        frequency_penalty = 0,
        presence_penalty = 0,
        stop = ['#', ';']
)

In [99]:
response['choices'][0]['text']


' City, SUM(Sales) AS Total_Sales\nFROM Sales\nGROUP BY City'

In [100]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(' '):
        query = 'SELECT' + query
    return query    

In [101]:
handle_response(response)

'SELECT City, SUM(Sales) AS Total_Sales\nFROM Sales\nGROUP BY City'

In [102]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-04-28 10:36:20,792 INFO sqlalchemy.engine.Engine SELECT City, SUM(Sales) AS Total_Sales
FROM Sales
GROUP BY City
2023-04-28 10:36:20,797 INFO sqlalchemy.engine.Engine [generated in 0.00535s] ()


In [90]:
result.all()

[('Aaarhus', 100595.54999999999),
 ('Allentown', 122138.14000000001),
 ('Barcelona', 78411.86000000002),
 ('Bergamo', 137955.72000000003),
 ('Bergen', 111640.28),
 ('Boras', 134259.33000000002),
 ('Boston', 154069.65999999997),
 ('Brickhaven', 165255.20000000004),
 ('Bridgewater', 101894.79000000001),
 ('Brisbane', 50218.51000000001),
 ('Bruxelles', 74972.52),
 ('Burbank', 46084.63999999999),
 ('Burlingame', 120783.07),
 ('Cambridge', 139243.99999999994),
 ('Charleroi', 33440.1),
 ('Chatswood', 151570.98000000004),
 ('Cowes', 78240.83999999998),
 ('Dublin', 57756.43),
 ('Espoo', 113961.14999999997),
 ('Frankfurt', 85171.58999999998),
 ('Gensve', 117713.55999999998),
 ('Glen Waverly', 64591.46000000001),
 ('Glendale', 66423.77),
 ('Graz', 52263.899999999994),
 ('Helsinki', 111250.37999999996),
 ('Kobenhavn', 145041.6),
 ('Koln', 100306.58),
 ('Las Vegas', 82751.08000000002),
 ('Lille', 69052.41),
 ('Liverpool', 118008.26999999999),
 ('London', 124823.54),
 ('Los Angeles', 48048.46),
 ('

### Production code